참고:
- https://doheon.github.io/%EC%BD%94%EB%93%9C%EA%B5%AC%ED%98%84/nlp/ci-chatbot-post/

오픈된 데이터와 pytorch의 nn.Transformer를 이용하여 간단한 챗봇을 제작해보았다.

아래의 강의를 참고하였으며 여기서 Tensorflow로 구현된 것을 참고하여 Pytorch로 다시 구현했다.

Code: https://github.com/Doheon/Chatbot-Transformer

Reference: https://wikidocs.net/89786

Data: https://github.com/songys/Chatbot_data

# Data Load

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data1 = pd.read_csv("AllData.csv")
data1

,req,res
0,너 좋아하는 차 종류 있어?,무슨 차? 자동차? 마시는 차?
1,ㅋㅋ 마시는 차 말한 거야!,"아하 나 둥글레, 옥수수, 보리차 좋아해"
2,완전 곡물류 좋아하네 ㅋㅋ,야쓰 끓이기 귀찮아서 냉침해 먹어
3,그럼 오래 걸리지 않아?,끓이는 것보다는 훨씬 오래 걸리지 ㅠ
4,근데 냉침 하는 것도 귀찮겠다 ㅜㅠ,응! 그래서 매일은 안 먹고 가끔 마셔
...,...,...
88969,아르바이트 면접 언제야?,내일 아르바이트 면접이야!
88970,무슨 아르바이트 지원했어?,나 카페 아르바이트 지원했어
88971,카페는 경력자 원하지 않아?,맞아 나 전에 카페 경력 일년 정도 있어서 괜찮을걸?
88972,아 맞네 ㅋㅋ 너 카페 일했었네,응 근데 까먹었긴 한데 일 배우면 또 다해


In [ ]:
data1.rename(columns={"req":"Q", "res":"A"}, inplace=True)
data1["label"] = 0

In [ ]:
data2 = pd.read_csv("ChatbotData.csv")
data2.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
train_data = pd.concat([data2, data1], axis=0)
train_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
88969,아르바이트 면접 언제야?,내일 아르바이트 면접이야!,0
88970,무슨 아르바이트 지원했어?,나 카페 아르바이트 지원했어,0
88971,카페는 경력자 원하지 않아?,맞아 나 전에 카페 경력 일년 정도 있어서 괜찮을걸?,0
88972,아 맞네 ㅋㅋ 너 카페 일했었네,응 근데 까먹었긴 한데 일 배우면 또 다해,0


In [ ]:
train_data.reset_index(drop=True, inplace=True)

# Preprocessing

In [ ]:
train_data["Q"] = train_data["Q"].str.split("/").str[0]
train_data["A"] = train_data["A"].str.split("/").str[0]

In [ ]:
train_data.shape

(100797, 3)

In [ ]:
train_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
100792,아르바이트 면접 언제야?,내일 아르바이트 면접이야!,0
100793,무슨 아르바이트 지원했어?,나 카페 아르바이트 지원했어,0
100794,카페는 경력자 원하지 않아?,맞아 나 전에 카페 경력 일년 정도 있어서 괜찮을걸?,0
100795,아 맞네 ㅋㅋ 너 카페 일했었네,응 근데 까먹었긴 한데 일 배우면 또 다해,0


## 한글 맞춤법 검사

### stripping illegal characters out of xml

In [ ]:
!git clone https://github.com/ssut/py-hanspell.git

In [ ]:
from hanspell import spell_checker

In [ ]:
spell_checker.check(u'안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.').checked

'안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.'

In [ ]:
# train_data["Q"] = train_data["Q"].str.replace(r"[<&>'\"]", "")
# train_data["A"] = train_data["A"].str.replace(r"[<&>'\"]", "")

In [ ]:
# train_data["Q"] = train_data["Q"].apply(lambda text: spell_checker.check(str(text)).checked)
# train_data["A"] = train_data["A"].apply(lambda text: spell_checker.check(str(text)).checked)
# train_data

### ? ! . 양옆에 공백을 추가

In [ ]:
questions = []
for sentence in train_data["Q"]:
    sentence = re.sub(r"([?.!,])", r" \1 ", str(sentence))
    sentence = sentence.strip()
    questions.append(sentence)

In [ ]:
answers = []
for sentence in train_data["A"]:
    sentence = re.sub(r"([?.!,])", r" \1 ", str(sentence))
    sentence = sentence.strip()
    answers.append(sentence)

데이터를 list에 저장한다. !?. 와 같은 기호들은 공백을 추가해 준다.

In [ ]:
print(questions[:5])
print(answers[:5])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']


In [ ]:
import sentencepiece as spm

In [ ]:
with open('all.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(questions))
    f.write('\n'.join(answers))

In [ ]:
corpus = "all.txt"
prefix = "chatbot"
# vocab_size = 16000
vocab_size = 20000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

KeyboardInterrupt: 

In [ ]:
vocab_file = "chatbot.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)
line = "안녕하세요 만나서 반갑습니다"
pieces = vocab.encode_as_pieces(line)
ids = vocab.encode_as_ids(line)


print(line)
print(pieces)
print(ids)

안녕하세요 만나서 반갑습니다
['▁안녕하세요', '▁만나서', '▁반갑', '습니다']
[16724, 2315, 10071, 992]


In [ ]:
# 최대 길이를 40으로 정의
MAX_LENGTH = 40
START_TOKEN = [2]
END_TOKEN = [3]

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    zeros1 = np.zeros(MAX_LENGTH, dtype=int)
    zeros2 = np.zeros(MAX_LENGTH, dtype=int)
    sentence1 = START_TOKEN + vocab.encode_as_ids(sentence1) + END_TOKEN
    zeros1[:len(sentence1)] = sentence1[:MAX_LENGTH]

    sentence2 = START_TOKEN + vocab.encode_as_ids(sentence2) + END_TOKEN
    zeros2[:len(sentence2)] = sentence2[:MAX_LENGTH]

    tokenized_inputs.append(zeros1)
    tokenized_outputs.append(zeros2)
  return tokenized_inputs, tokenized_outputs

In [ ]:
questions_encode, answers_encode = tokenize_and_filter(questions, answers)

In [ ]:
print(questions_encode[0])
print(answers_encode[0])

[    2  1793 28610  2040    27     3     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
[    2  8386   198 25424     7     3     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]


In [ ]:
from torch.utils.data import Dataset, DataLoader

class SequenceDataset(Dataset):
    def __init__(self, questions, answers):
        questions = np.array(questions)
        answers = np.array(answers)
        self.inputs = questions
        self.dec_inputs = answers[:,:-1]
        self.outputs = answers[:,1:]
        self.length = len(questions)

    def __getitem__(self,idx):
        return (self.inputs[idx], self.dec_inputs[idx], self.outputs[idx])

    def __len__(self):
        return self.length

BATCH_SIZE = 256
dataset = SequenceDataset(questions_encode, answers_encode)
dataloader = DataLoader(dataset, shuffle=True, batch_size=BATCH_SIZE)

# Define Model

In [ ]:
from torch.nn import Transformer
from torch import nn
import torch
import math

class TFModel(nn.Module):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TFModel, self).__init__()
        self.transformer = Transformer(ninp, nhead, dim_feedforward=nhid, num_encoder_layers=nlayers, num_decoder_layers=nlayers,dropout=dropout)
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        self.encoder = nn.Embedding(ntoken, ninp)

        self.pos_encoder_d = PositionalEncoding(ninp, dropout)
        self.encoder_d = nn.Embedding(ntoken, ninp)

        self.ninp = ninp
        self.ntoken = ntoken

        self.linear = nn.Linear(ninp, ntoken)
        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, tgt, srcmask, tgtmask, srcpadmask, tgtpadmask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)

        tgt = self.encoder_d(tgt) * math.sqrt(self.ninp)
        tgt = self.pos_encoder_d(tgt)


        output = self.transformer(src.transpose(0,1), tgt.transpose(0,1), srcmask, tgtmask, src_key_padding_mask=srcpadmask, tgt_key_padding_mask=tgtpadmask)
        output = self.linear(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def gen_attention_mask(x):
    mask = torch.eq(x, 0)
    return mask

# Device

In [ ]:
# device = torch.device("cuda", index=1)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Init Model

In [ ]:
model = TFModel(vocab_size+7, 256, 8, 512, 4, 0.1).to(device)

# Hyper Parameter

In [ ]:
lr = 1e-4
epoch = 100
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
import wandb

wandb.login()
wandb.init(project="transformer")
wandb.config = {
    "epochs": 100,
    "batch_size": dataloader.batch_size,
    "learning_rate": optimizer.param_groups[0]['lr'],
}
wandb.watch(model)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


[]

In [ ]:
from tqdm import tqdm

best_loss = float("inf")
model.train()
for i in range(epoch):


    batchloss = 0.0
    progress = tqdm(dataloader)
    for (inputs, dec_inputs, outputs) in progress:
        optimizer.zero_grad()
        src_mask = model.generate_square_subsequent_mask(MAX_LENGTH).to(device)
        src_padding_mask = gen_attention_mask(inputs).to(device)
        tgt_mask = model.generate_square_subsequent_mask(MAX_LENGTH-1).to(device)
        tgt_padding_mask = gen_attention_mask(dec_inputs).to(device)

        result = model(inputs.to(device), dec_inputs.to(device), src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
        loss = criterion(result.permute(1,2,0), outputs.to(device).long())
        progress.set_description("{:0.3f}".format(loss))
        loss.backward()
        optimizer.step()
        batchloss += loss

    print("epoch:", i+1, "|", "loss:", batchloss.to(device).item()/len(dataloader))

    if (i+1) % 10 == 0: torch.save(model.state_dict(), f"chatbot_{i+1}.pth")
    if best_loss > batchloss.to(device).item():
        best_loss = batchloss.to(device).item()
        torch.save(model.state_dict(), f"chatbot_best.pth")

    wandb.log({
        "Epoch": i+1,
        "Learning Rate": optimizer.param_groups[0]['lr'],
        "Loss": round(batchloss.to(device).item()/len(dataloader), 3),
    })


  0%|          | 0/394 [00:00<?, ?it/s]

c:\Users\user\.conda\envs\blessian\Lib\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched src_key_padding_mask and mask is deprecated. Use same type for both instead.
  warnings.warn(
c:\Users\user\.conda\envs\blessian\Lib\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
1.673: 100%|██████████| 394/394 [27:10<00:00,  4.14s/it]


epoch: 1 | loss: 2.1467241781012056


1.510: 100%|██████████| 394/394 [29:51<00:00,  4.55s/it]


epoch: 2 | loss: 1.5853728473489055


...

epoch: 14 | loss: 1.3317508600690038


1.232: 100%|██████████| 394/394 [26:57<00:00,  4.11s/it]


epoch: 15 | loss: 1.3142337702252538


1.242:  75%|███████▌  | 297/394 [19:36<06:24,  3.96s/it]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "chatbot_last.pth")

In [ ]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", str(sentence))
    sentence = sentence.strip()
    return sentence

def evaluate(model, sentence):
    sentence = preprocess_sentence(sentence)
    input = torch.tensor([START_TOKEN + vocab.encode_as_ids(sentence) + END_TOKEN]).to(device)
    output = torch.tensor([START_TOKEN]).to(device)

    # 디코더의 예측 시작
    model.eval()
    for i in range(MAX_LENGTH):
        src_mask = model.generate_square_subsequent_mask(input.shape[1]).to(device)
        tgt_mask = model.generate_square_subsequent_mask(output.shape[1]).to(device)

        src_padding_mask = gen_attention_mask(input).to(device)
        tgt_padding_mask = gen_attention_mask(output).to(device)

        predictions = model(input, output, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask).transpose(0,1)
        # 현재(마지막) 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = torch.LongTensor(torch.argmax(predictions.cpu(), axis=-1))


        # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if torch.equal(predicted_id[0][0], torch.tensor(END_TOKEN[0])):
            break

        # 마지막 시점의 예측 단어를 출력에 연결한다.
        # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
        output = torch.cat([output, predicted_id.to(device)], axis=1)

    return torch.squeeze(output, axis=0).cpu().numpy()

def predict(model, sentence):
    prediction = evaluate(model, sentence)
    predicted_sentence = vocab.Decode(list(map(int,[i for i in prediction if i < vocab_size+7])))

    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence

In [ ]:
from pathlib import Path

def test(dir_path):
    test_sentences = [
        "난 뭘 해야 할까?",
        "월말 프로젝트를 뭘로 해야할지 모르겠어",
        "점심 메뉴 뭐하지?",
        "회의를 아무리 해봐도 쓸만한 아이디어가 없어서 고민이야",
        "빨리 집에 가고 싶어"
    ]

    model_paths = Path(dir_path).glob("*.pth")
    for model_path in model_paths:
        model.load_state_dict(torch.load(str(model_path)))
        print("="*150)

        for sentence in test_sentences:
            print("-"*30, model_path.stem, "-"*30)
            predict(model, sentence)

In [ ]:
test("./")

------------------------------ chatbot_10 ------------------------------
Input: 난 뭘 해야 할까?
Output: 맞아 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
------------------------------ chatbot_10 ------------------------------
Input: 월말 프로젝트를 뭘로 해야할지 모르겠어
Output: 맞아 , 그래서 그 사람 많이 안 해
------------------------------ chatbot_10 ------------------------------
Input: 점심 메뉴 뭐하지?
Output: 좋은 사람 더 좋은 것 같아요 .
------------------------------ chatbot_10 ------------------------------
Input: 회의를 아무리 해봐도 쓸만한 아이디어가 없어서 고민이야
Output: 맞아 , 그래서 그런 것 같아 .
------------------------------ chatbot_10 ------------------------------
Input: 빨리 집에 가고 싶어
Output: 맞아 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
------------------------------ chatbot_100 ------------------------------
Input: 난 뭘 해야 할까?
Output: 그 마음 단단히 잡았으면 한 번씩 헤어져를 얻으면 대게 되는 거 필요 없을텐데 말이야 . . .
------------------------------ chatbot_100 ------------------------------
Input: 월말 프로젝트를 뭘로 해야할지 

In [ ]:
model.load_state_dict(torch.load("chatbot_last.pth"))
predict(model, "자소서 쓰기 너무 귀찮아 ㅠㅜ")

Input: 자소서 쓰기 너무 귀찮아 ㅠㅜ
Output: 아 . . . 그럼 그 마음 놓고 생활하는 건드릴 수 있는 게 아니라 따로 챙기는 건드릴 수 있는 거네 ?


'아 . . . 그럼 그 마음 놓고 생활하는 건드릴 수 있는 게 아니라 따로 챙기는 건드릴 수 있는 거네 ?'